In [5]:
!pip install -q pyspark numpy pandas pyarrow

from pyspark.sql import SparkSession, functions as F
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import Row

spark = SparkSession.builder.appName("AQI_Regression_Fast").getOrCreate()
spark.sparkContext.setLogLevel("WARN")

df = (spark.read.option("header", True).option("inferSchema", True).csv("/content/part-*.csv"))

aqi_cols = [c for c in ["NO2_AQI","O3_AQI","SO2_AQI","CO_AQI","PM2_5_AQI","PM10_AQI"] if c in df.columns]
df = df.withColumn("AQI_label", F.array_max(F.array(*[F.col(c).cast("double") for c in aqi_cols]))).dropna(subset=["AQI_label"])

feature_cols = [c for c in ["NO2_Mean","O3_Mean","SO2_Mean","CO_Mean","PM2_5_Mean","PM10_Mean"] if c in df.columns]
df = df.dropna(subset=feature_cols).select(*(feature_cols + ["AQI_label"])).sample(False, 0.3, 42).cache()

train, test = df.randomSplit([0.8, 0.2], seed=42)

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features_vec")
scaler = StandardScaler(inputCol="features_vec", outputCol="features_z", withMean=True, withStd=True)
lr = LinearRegression(featuresCol="features_z", labelCol="AQI_label", predictionCol="AQI_pred", maxIter=50, regParam=0.1, elasticNetParam=0.2)

model = Pipeline(stages=[assembler, scaler, lr]).fit(train)

pred = model.transform(test).cache()
rmse = RegressionEvaluator(labelCol="AQI_label", predictionCol="AQI_pred", metricName="rmse").evaluate(pred)
mae  = RegressionEvaluator(labelCol="AQI_label", predictionCol="AQI_pred", metricName="mae").evaluate(pred)
r2   = RegressionEvaluator(labelCol="AQI_label", predictionCol="AQI_pred", metricName="r2").evaluate(pred)
print(f"RMSE: {rmse:.4f}  MAE: {mae:.4f}  R2: {r2:.4f}")

coef_stage = model.stages[-1]
coef_df = spark.createDataFrame([Row(feature=f, coefficient=float(c)) for f, c in zip(feature_cols, coef_stage.coefficients.toArray())])
coef_df.write.mode("overwrite").parquet("/content/aqi_regression_coefficients.parquet")
pred.select(*feature_cols, "AQI_label", "AQI_pred").write.mode("overwrite").parquet("/content/aqi_regression_predictions.parquet")
spark.createDataFrame([("rmse", float(rmse)), ("mae", float(mae)), ("r2", float(r2))], ["metric","value"]).write.mode("overwrite").parquet("/content/aqi_regression_metrics.parquet")

pred.select(*feature_cols, "AQI_label", "AQI_pred").show(10, truncate=False)


RMSE: 11.6549  MAE: 7.5212  R2: 0.6255
+---------+--------+--------+--------+---------+-------------------+
|NO2_Mean |O3_Mean |SO2_Mean|CO_Mean |AQI_label|AQI_pred           |
+---------+--------+--------+--------+---------+-------------------+
|-0.279167|0.03125 |0.0125  |0.137417|30.0     |30.30440163241441  |
|-0.154167|0.010125|0.021739|0.1     |18.0     |2.206747638298644  |
|0.0      |0.007542|0.0     |0.1     |12.0     |-1.0631346526626828|
|0.0      |0.012   |0.0     |0.066667|15.0     |4.741218782912227  |
|0.0      |0.012167|0.125   |0.404167|14.0     |6.312568235434199  |
|0.0      |0.014   |0.0     |0.05    |16.0     |7.338728969426327  |
|0.0      |0.0145  |0.55    |0.008333|19.0     |8.140627885372538  |
|0.0      |0.015375|0.8     |0.2     |16.0     |10.166945865663262 |
|0.0      |0.016083|0.033333|0.1     |18.0     |10.31780229036676  |
|0.0      |0.016125|0.0     |0.120833|19.0     |10.434944697287406 |
+---------+--------+--------+--------+---------+----------------